In [81]:
import random as rand

In [82]:
# Variables importantes
numero_poblacion = 10
generaciones = 50
mochila = 10
peso_mochila = 30
probabilidad_cruza = 0.85
probabilidad_mutacion = 0.1
cruza_uniforme = 0.5

# Decoy detonators, love potion (min 3), extendable ears, skiving snackbox(min 2), fever fudge, puking pastilles, nosebleed nougat
ganancia = [10, 8, 12, 6, 3, 2, 2]
peso = [4, 2, 5, 5, 2, 1.5, 1]

In [83]:
def calcular_peso_individuo(individuo):
    peso_total = 0

    for i in range(7):
        peso_total += individuo[i] * peso[i]

    return peso_total

In [84]:
def calcular_peso_poblacion(poblacion):
    peso = []
    for individuo in poblacion:
        peso.append(calcular_peso_individuo(individuo))
    return peso

In [85]:
# Generar población inicial
def poblacion(individuos):
    
    poblacion = []
    j = 0
    
    for i in range (individuos):
        
        individuo = [0,0,0,0,0,0,0]
        
        while j <= 6:
            if j == 1:
                individuo[1] = rand.randint(3,10)
            elif j == 3:
                individuo[3] = rand.randint(2,10)
            else:
                individuo[j] = rand.randint(0,10)
            j += 1 
        poblacion.append(individuo)
        j = 0
    
    return poblacion

In [86]:
def aptitud(poblacion):
    suma_total = 0
    suma_indv = [0]*numero_poblacion
    j = 0

    for individuo in poblacion:
        for i in range(7):
            suma_total += individuo[i] * ganancia[i]
            suma_indv[j] += individuo[i] * ganancia[i]
        j += 1

    # Calcular aptitud
    aptitud = []
    aptitud_acumulada = []

    for i in range (len(poblacion)):
        aptitud.append(suma_indv[i]/suma_total)
        
        if i == 0:
            aptitud_acumulada.append(aptitud[i])
        else:
            aptitud_acumulada.append(aptitud_acumulada[i-1]+aptitud[i])

    return aptitud_acumulada

In [87]:
def aptitud_indv(individuo):
    suma_total = 0

    for i in range(7):
        suma_total += individuo[i] * ganancia[i]

    return suma_total

In [88]:
def mutacion(hijo1,hijo2):
    
    var_aleatoria1 = []
    var_aleatoria2 = []
    condicion = True

    for i in range(7):
        var_aleatoria1.append(rand.uniform(0,1))
        var_aleatoria2.append(rand.uniform(0,1))

    for i in range(7):
        if probabilidad_mutacion >= var_aleatoria1[i]:
            if i == 1:
                hijo1[i] = rand.randint(3,10)
            elif i == 3:
                hijo1[i] = rand.randint(2,10)
            else:
                hijo1[i] = rand.randint(0,10)
        if probabilidad_mutacion >= var_aleatoria2[i]:
            if i == 1:
                hijo2[i] = rand.randint(3,10)
            elif i == 3:
                hijo2[i] = rand.randint(2,10)
            else:
                hijo2[i] = rand.randint(0,10)
    
    if calcular_peso_individuo(hijo1) > peso_mochila or calcular_peso_individuo(hijo2) > peso_mochila:
        condicion = False

    return hijo1, hijo2, condicion

In [89]:
def cruza(padres):
    
    var_aleatoria = []
    hijo1 = []
    hijo2 = []
    
    for i in range(7):
        var_aleatoria.append(rand.uniform(0,1))
    
    for i in range(7):
        if cruza_uniforme >= var_aleatoria[i]:
            hijo1.append(padres[0][i])
            hijo2.append(padres[1][i])
        else:
            hijo1.append(padres[1][i])
            hijo2.append(padres[0][i])
    
    return mutacion(hijo1, hijo2)

In [90]:
def ruleta(aptitud, poblacion):
    
    #print(aptitud)
    
    padres = []
    random1 = rand.uniform(0,1)
    #print(random1)    
    
    for i, apt in enumerate(aptitud):
        if apt >= random1 and poblacion[i] not in padres:
            padres.append(poblacion[i])
            break
            
    while len(padres) != 2:
        random2 = rand.uniform(0,1)
        for i, apt in enumerate(aptitud):
            if apt >= random2 and poblacion[i] not in padres:
                padres.append(poblacion[i])
                break

    #print("Padres: ", padres)
        
    random_cruza = rand.uniform(0,1)
    if probabilidad_cruza >= random_cruza:
        return padres, True
    else:
        #print("lol q mal")
        return padres, False

In [91]:
def padre_mas_debil(hijos, padres):
    aptitudes_hijos = [aptitud_indv(hijos[0]), aptitud_indv(hijos[1])]
    aptitudes_padres = [aptitud_indv(padres[0]), aptitud_indv(padres[1])]
    
    if max(aptitudes_hijos) < min(aptitudes_padres):
        return padres
    elif min(aptitudes_hijos) > max(aptitudes_padres):
        return hijos
    elif max(aptitudes_hijos) == max(aptitudes_padres) and min(aptitudes_hijos) == min(aptitudes_padres):
        random = rand.randint(0,1)
        if random == 0:
            return padres
        else:
            return hijos
    else:
        individuos_optimos = ["lol","no se declarar arreglos"]

        for i in range(2):
            for j in range(2):
                if aptitudes_hijos[i] > aptitudes_padres[j]:
                    individuos_optimos[i] = hijos[i]
                elif aptitudes_hijos[i] < aptitudes_padres[j]:
                    individuos_optimos[i] = padres[j]
                else:
                    random = rand.randint(0,1)
                    if random:
                        individuos_optimos[i] = hijos[i]
                    else:
                        individuos_optimos[i] = padres[j]

        return individuos_optimos

In [96]:

ind = 1

while ind:
    pob = poblacion(numero_poblacion)
    pesos = calcular_peso_poblacion(pob)
    for i in pesos:
        if i <= peso_mochila:
            ind = 0
        else:
            ind = 1

for i in range(generaciones):
    print("Generación: ", i+1)
    print("Población actual: ",pob)
    apt = aptitud(pob)
    x = 0
    nueva_poblacion = []
    while x < numero_poblacion/2:

        padres, flag = ruleta(apt, pob)

        if flag:
            hijo1, hijo2, flag2 = cruza(padres)
            while flag2 is False:
                padres, flag = ruleta(apt, pob)
                hijo1, hijo2, flag2 = cruza(padres) 
            hijos = [hijo1, hijo2]
            individuos_optimos = padre_mas_debil(hijos, padres)
            nueva_poblacion.append(individuos_optimos[0])
            nueva_poblacion.append(individuos_optimos[1])

        else:
            nueva_poblacion.append(padres[0])
            nueva_poblacion.append(padres[1])

        x += 1

    print("nueva_poblacion: ", nueva_poblacion)
    pob = nueva_poblacion

Generación:  1
Población actual:  [[5, 3, 7, 8, 7, 9, 4], [6, 5, 10, 2, 1, 8, 0], [2, 6, 0, 8, 9, 10, 3], [8, 7, 3, 9, 3, 5, 3], [4, 3, 4, 10, 5, 3, 0], [7, 3, 1, 7, 6, 6, 10], [0, 9, 4, 6, 1, 1, 1], [7, 7, 2, 4, 6, 9, 9], [10, 10, 7, 5, 8, 2, 8], [1, 4, 1, 2, 0, 1, 0]]
nueva_poblacion:  [[1, 4, 1, 2, 0, 1, 1], [1, 4, 1, 2, 0, 1, 0], [0, 5, 1, 2, 1, 1, 0], [1, 4, 1, 2, 0, 1, 0], [6, 5, 10, 2, 1, 8, 0], [1, 4, 1, 2, 0, 1, 0], [1, 4, 1, 2, 0, 1, 0], [0, 9, 4, 6, 1, 1, 1], [6, 5, 10, 2, 1, 8, 0], [1, 4, 1, 2, 0, 1, 0]]
Generación:  2
Población actual:  [[1, 4, 1, 2, 0, 1, 1], [1, 4, 1, 2, 0, 1, 0], [0, 5, 1, 2, 1, 1, 0], [1, 4, 1, 2, 0, 1, 0], [6, 5, 10, 2, 1, 8, 0], [1, 4, 1, 2, 0, 1, 0], [1, 4, 1, 2, 0, 1, 0], [0, 9, 4, 6, 1, 1, 1], [6, 5, 10, 2, 1, 8, 0], [1, 4, 1, 2, 0, 1, 0]]
nueva_poblacion:  [[1, 4, 1, 2, 0, 1, 0], [1, 4, 1, 2, 0, 1, 1], [1, 4, 1, 2, 0, 1, 1], [1, 4, 1, 2, 0, 1, 0], [1, 4, 1, 2, 0, 1, 0], [0, 5, 1, 2, 1, 1, 0], [1, 4, 1, 2, 0, 1, 0], [6, 5, 10, 2, 1, 8, 0], [1, 4, 

In [93]:
pob = poblacion(numero_poblacion)
print("Población inicial: ", pob)
print("Peso población inicial: ", calcular_peso_poblacion(pob))
print()

for i in range(3):
    

_IncompleteInputError: incomplete input (1962882445.py, line 7)